In [1]:
from transformers import BertTokenizer, BertConfig, TFBertModel
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [33]:
MAX_LEN=32

In [34]:
model_name = 'bert-base-uncased'

# Defining RoBERTa tokinizer
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True)

In [35]:
tokenizer.tokenize('Testing something new. Hello dear')

['testing', 'something', 'new', '.', 'hello', 'dear']

In [4]:
train_df['num_of_words'] = train_df.apply(lambda r: min(256, len(r['comment_text'].split(' '))), axis=1)

NameError: name 'train_df' is not defined

In [5]:
train_df['num_of_words'].hist(bins=32)

NameError: name 'train_df' is not defined

In [38]:
len(train_df[train_df['num_of_words'] == 256])

10781

In [35]:
len(train_df)

286692

In [36]:
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')

In [37]:
ids, masks, segments = tokenize([
    'First something new',
    'Another something'
], tokenizer)

100%|██████████| 2/2 [00:00<00:00, 2712.13it/s]


In [38]:
ids.shape, masks.shape, segments.shape

((2, 32), (2, 32), (2, 32))

In [39]:
config = BertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [40]:
train_df = pd.read_csv('../dataset/train_augmented.csv', index_col='idx')

In [41]:
test = pd.read_csv('../dataset/test_processed.csv')
test_df = test[test['class'] != -1]

In [21]:
def preapre_environment():
    physical_devices = tf.config.list_physical_devices('GPU')
    # print(physical_devices)

    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        # print('Tensorflow sets memory gropth to True')
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

In [22]:
preapre_environment()

In [42]:
train_ids, train_masks, train_segment = tokenize(train_df["comment_text"].values, tokenizer)

100%|██████████| 286692/286692 [05:22<00:00, 888.11it/s] 


In [27]:
np.save('../dataset/train_ids.npy', train_ids)
np.save('../dataset/train_masks.npy', train_masks)
np.save('../dataset/train_segment.npy', train_segment)

In [43]:
np.save('../dataset/train_ids_32.npy', train_ids)
np.save('../dataset/train_masks_32.npy', train_masks)
np.save('../dataset/train_segment_32.npy', train_segment)

In [ ]:
train_ids = np.load('../dataset/train_ids.npy')
train_masks = np.load('../dataset/train_masks.npy')
train_segment = np.load('../dataset/train_segment.npy')

In [44]:
test_ids, test_masks, test_segment =  tokenize(test_df["comment_text"].values, tokenizer)

100%|██████████| 63978/63978 [01:16<00:00, 838.40it/s] 


In [28]:
np.save('../dataset/test_ids.npy', test_ids)
np.save('../dataset/test_masks.npy', test_masks)
np.save('../dataset/test_segment.npy', test_segment)

In [45]:
np.save('../dataset/test_ids_32.npy', test_ids)
np.save('../dataset/test_masks_32.npy', test_masks)
np.save('../dataset/test_segment_32.npy', test_segment)

In [ ]:
test_ids = np.load('../dataset/test_ids.npy')
test_masks = np.load('../dataset/test_masks.npy')
test_segment = np.load('../dataset/test_segment.npy')

In [26]:
y_train = train_df["class"].values
y_test = test_df["class"].values

In [50]:
input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

last_hidden_state, pooler_output = transformer_model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids)

x = tf.keras.layers.Dense(256, activation = "relu")(pooler_output)
x = tf.keras.layers.Dense(1, activation = "sigmoid")(x)

model = tf.keras.Model(inputs=[input_ids, input_mask, segment_ids], outputs = x)

for layer in model.layers[:4]:
    layer.trainable = False

In [51]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 32)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 32)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   ((None, 32, 768), (N 109482240   input_ids[0][0]                  
____________________________________________________________________________________________

In [52]:
model.compile(tf.keras.optimizers.Adam(lr=2e-6), loss="binary_crossentropy", metrics=["accuracy"])

In [59]:
MAX_SAMPLES = -1 #16 * 64

In [61]:
history = model.fit(
    (train_ids[:MAX_SAMPLES], train_masks[:MAX_SAMPLES], train_segment[:MAX_SAMPLES]), y_train[:MAX_SAMPLES],
    epochs=1,
    batch_size=128
)

2240/2240 [==============================] - 587s 262ms/step - loss: 0.1106 - accuracy: 0.9696


In [65]:
model.save_weights('model_longer.h5')

In [71]:
test_preds = model.predict((test_ids, test_masks, test_segment))

In [70]:
model.evaluate((test_ids, test_masks, test_segment), y_test)

TypeError: evaluate() got an unexpected keyword argument 'metric'

In [64]:
tf.keras.metrics.BinaryAccuracy(0.5)

In [74]:
!pip install scikit-learn

     |████████████████████████████████| 6.8 MB 3.5 MB/s eta 0:00:01


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, test_preds)
accuracy_scores = []
for thresh in thresholds:
    accuracy_scores.append(accuracy_score(y_test, 
                                         [1 if m > thresh else 0 for m in test_preds]))

accuracies = np.array(accuracy_scores)
max_accuracy = accuracies.max() 
max_accuracy_threshold =  thresholds[accuracies.argmax()]